<a href="https://colab.research.google.com/github/suafle/minitarea4/blob/master/minitarea4_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CD2201: Ciencia de Datos en Astronomía**
**Profesores**: Pía Cortés, Daniela Barrientos, Matías Suazo, Matías Mattamala




**Minitarea 4**: Uso de scikit-learn

**Objetivos:**

1.   Poner en práctica el uso de todas las librerías de python anteriormente vistas.
2.   Utilizar el paquete scikit-learn para regresión y clasificación.
3.   Medir el desempeño de los algoritmos a utilizar.

**Instrucciones:**

1.   Leer cuidadosamente lo que sigue.
2.   


##**1. Cargar datos**

Para esta minitarea trabajaremos con datos fotométricos de galaxias y estrellas. Esta tarea cuenta con dos objetivos: aplicar algoritmos de machine learning tanto en regresión como en clasificación, como evaluar el desempeño de los algoritmos a utilizar.

Comencemos por cargar los datos que han sido provistos a través de material docente (**datos_ml.csv**), el archivo debería estar en su Google Drive personal. Debemos montar nuestro Google Drive primero para luego acceder a los archivos en él. El archivo se puede abrir utilizando la función **read_csv** del paquete **pandas**.


Este archivo contiene solo fotometría en la banda r:


*   psfMag_r = Magnitud de la PSF
*   fiberMag_r = Magnitud en un radio de fibra de 3 segundos de arco
*   fiber2Mag_r = Magnitud en un radio de fibra de 3 segundos de arco
*   petroMag_r = Magnitud petrosiana (considerando un radio especial llamado radio petrosiano)
*   deVMag_r = Magnitud de Vaucouleurs (Se ajusta una función de De Vaucouvelurs y se calcula la magnitud suponiendo que el objeto obedece esta ley) https://en.wikipedia.org/wiki/De_Vaucouleurs%27_law
*   expMag_r = Magnitud asumiendo un ajuste exponencial
*   modelMag_u	= deVMag_r o expMag_r, dependiendo de cual de las dos resulta mejor ajustadas.
*   cModelMag_r	= magnitud creada a partir de deVMag_r y expMag_r







In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
datos = pd.read_csv('/content/drive/My Drive/datos_ml.csv')

In [ ]:
datos

,Unnamed: 0,psfMag_r,fiberMag_r,fiber2Mag_r,petroMag_r,deVMag_r,expMag_r,modelMag_r,cModelMag_r,class
0,0,20.44020,20.78680,21.41974,19.63153,19.44181,19.65987,19.44181,19.49099,b'GALAXY'
1,1,20.59600,21.05023,21.52178,20.43809,20.55688,20.56577,20.55688,20.55688,b'STAR'
2,2,21.20326,21.68447,22.09465,21.44075,21.21554,21.21564,21.21554,21.21554,b'STAR'
3,3,21.11595,21.52400,22.05365,20.81025,20.74396,20.82218,20.74391,20.74396,b'GALAXY'
4,4,21.80693,22.16929,22.75302,20.35699,20.21300,20.49648,20.21300,20.21300,b'GALAXY'
...,...,...,...,...,...,...,...,...,...,...
99995,99995,19.40304,19.73992,20.21592,19.46891,19.41325,19.41312,19.41312,19.41312,b'STAR'
99996,99996,15.89951,16.22320,16.70072,15.95597,15.90502,15.90497,15.90502,15.90502,b'STAR'
99997,99997,18.95202,19.27494,19.75539,19.00164,18.95163,18.95174,18.95164,18.95163,b'STAR'
99998,99998,19.62088,19.25105,19.98588,17.62660,17.14137,17.67299,17.67299,17.57948,b'GALAXY'


In [ ]:
from astroquery.sdss import SDSS

data = SDSS.query_sql("""SELECT TOP 100000
                      p.psfMag_r, p.fiberMag_r, p.fiber2Mag_r, p.petroMag_r, 
                      p.deVMag_r, p.expMag_r, p.modelMag_r, p.cModelMag_r, 
                      s.class
                      
                      FROM PhotoObjAll AS p JOIN specObjAll s ON s.bestobjid = p.objid
                      
                      WHERE p.mode = 1 AND s.sciencePrimary = 1 AND p.clean = 1 AND s.class != 'QSO'
                      
                      """)

/usr/local/lib/python3.6/dist-packages/astroquery/sdss/core.py:865: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  comments='#'))


In [ ]:
print(type(data))

t = data.to_pandas()

t.to_csv('datos_ml.csv',index=False)

NameError: ignored

In [ ]:
data.write('datos_ml.fits')

In [ ]:
!rm datos_ml.fits

In [ ]:
from google.colab import files

In [ ]:
files.download('datos_ml.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>